In [1]:
%autoreload 2
import sys
sys.path.append('C:\\code\\qdc2')
import numpy as np 
import matplotlib.pyplot as plt
from qdc.grating.grating_sim import GratingSim1D

In [53]:
sim = GratingSim1D(
    Nx          = 2**14,      # grid points
    Lx          = 8e-3,       # 8 mm
    wl0         = 808e-9,     # 808 nm blaze
    Dwl         = 80e-9,     # ±50 nm span
    N_wl        = 131,         # 11 wavelengths  (odd!)
    waist       = 3000e-6,      # 40 µm beam waist
    x0          = 0,
    blaze_angle = 0.1,       # blaze angle [rad]
    f           = 0.2,         # lens focal length [m]
    spectrum='flat', 
)

# Numerical patterns (as before)
x_det, I_class_num = sim.classical_pattern()
_,     I_spdc_num  = sim.spdc_pattern()

# Analytic predictions
_, I_class_anal = sim.classical_pattern_analytic()
_, I_spdc_anal  = sim.spdc_pattern_analytic()

# Convert x_det to diffraction order axis
m_vals = sim.diffraction_orders(x_det)

# Plot both numerical and analytic together
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(m_vals, I_class_num,  'b-', label="Numerical Classical")
print('done classical numerical')
ax.plot(m_vals, I_class_anal, 'b--', label="Analytic Classical")
print('done classical analytic')
ax.plot(m_vals, I_spdc_num,   'r-', label="Numerical SPDC")
print('done spdc numerical')
ax.plot(m_vals, I_spdc_anal,  'r--', label="Analytic SPDC")
print('done spdc analytic')
ax.set_yscale('log')
ax.set_xlabel("diffraction order $m$")
ax.set_ylabel("normalized intensity (log)")
# ax.set_ylim(1e-10, 1e4)
ax.set_title(f"{sim.N_wl} wavelengths")
ax.legend()
fig.tight_layout()
fig.show()

done classical numerical


ValueError: x and y must have same first dimension, but have shapes (16384,) and (2, 16384)

In [37]:
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(m_vals, I_class_num,  'b-', label="Numerical Classical")
print('done classical numerical')
ax.plot(m_vals, I_class_anal, 'b--', label="Analytic Classical")
print('done classical analytic')
ax.plot(m_vals, I_spdc_num,   'r-', label="Numerical SPDC")
print('done spdc numerical')
ax.plot(m_vals, I_spdc_anal,  'r--', label="Analytic SPDC")
print('done spdc analytic')
ax.set_yscale('log')
ax.set_xlabel("diffraction order $m$")
ax.set_ylabel("normalized intensity (log)")
ax.set_ylim(1e-10, 1e4)
ax.set_title(f"{sim.N_wl} wavelengths")
ax.legend()
fig.tight_layout()
fig.show()

done classical numerical
done classical analytic
done spdc numerical
done spdc analytic


In [15]:
print((x_det[1] - x_det[0])*I_class.sum())
print((x_det[1] - x_det[0])*I_spdc.sum())
print(I_spdc.max())
print(I_class.max())

0.9999861542040619
0.9999999329528425
221.18034185382217
318.74303467716913


In [7]:
from qdc.grating.grating_sim import gaussian, blazed_phase

if True:
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(sim.grating_phase, '--.')
    # ax.set_xlim([0, 1400])
    fig.show()
 
if True:
    E = gaussian(sim.x, sim.waist, sim.x0).astype(np.complex128)  
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot((np.abs(E))**2)
    fig.show()